Step 1: Dataset preparation

In [32]:
import json, random, os

os.makedirs("/content/drive/MyDrive/bfsi/data", exist_ok=True)

loan_amounts = ["1 lakh", "2 lakh", "3 lakh", "5 lakh", "10 lakh"]
tenures = ["12 months", "24 months", "36 months", "48 months"]
rates = ["9%", "10%", "11%", "12%", "13%"]

dataset = []

def add(inst, inp, out):
    dataset.append({
        "instruction": inst,
        "input": inp,
        "output": out
    })

# 1. Loan Eligibility (30)
for i in range(30):
    add(
        "Check loan eligibility",
        f"My salary is ₹{random.randint(20000,80000)} per month",
        "Based on your income details, you may be eligible for a personal loan. Final eligibility depends on credit score, repayment history, and internal policy checks. Kindly proceed with the application for confirmation."
    )

# 2. EMI Details (30)
for i in range(30):
    add(
        "Explain EMI details",
        f"Loan {random.choice(loan_amounts)} for {random.choice(tenures)}",
        "Your EMI will depend on the loan amount, tenure, and applicable interest rate. Once processed, the EMI schedule will be shared through your registered email and app dashboard."
    )

# 3. Interest Rates (30)
for i in range(30):
    add(
        "Provide interest rate information",
        "What is the interest rate for personal loans?",
        "Personal loan interest rates vary based on customer profile and credit assessment. Current rates generally start from competitive market levels. Please check the latest offers in your account portal."
    )

# 4. Payment Status (25)
for i in range(25):
    add(
        "Check payment status",
        "Has my EMI been received?",
        "You can verify your recent payment status in the transaction history section of the mobile app. If the payment was successful, it will reflect within 24 hours."
    )

# 5. Penalties (20)
for i in range(20):
    add(
        "Explain late payment penalty",
        "What happens if I miss my EMI?",
        "Delayed payments may attract late fees and additional interest as per policy. We recommend paying dues before the due date to avoid penalties and credit score impact."
    )

# 6. Transaction Queries (25)
for i in range(25):
    add(
        "Help with transaction issue",
        "My payment is not reflected",
        "If your payment is not visible, please allow standard processing time. If the issue persists beyond 24 hours, kindly contact customer support with the transaction reference number."
    )

# 7. Account Support (20)
for i in range(20):
    add(
        "Assist with account support",
        "How do I update my mobile number?",
        "You may update your registered mobile number through the profile section after completing OTP verification. For assistance, contact support."
    )

# 8. Insurance Queries (20)

for i in range(20):
    add(
        "Provide insurance policy details",
        "What does my insurance cover?",
        "Your policy coverage details are listed in the policy document. Please review the terms and conditions to understand inclusions, exclusions, and claim procedures."
    )

# Save file
path = "/content/drive/MyDrive/bfsi/data/alpaca_dataset.json"
with open(path, "w") as f:
    json.dump(dataset, f, indent=2)

print("Dataset saved at:", path)
print("Total samples:", len(dataset))


Dataset saved at: /content/drive/MyDrive/bfsi/data/alpaca_dataset.json
Total samples: 200


Similarity Engine

In [33]:
!pip install -q sentence-transformers faiss-cpu


Load Dataset

In [34]:
import json
DATA_PATH = "/content/drive/MyDrive/bfsi/data/alpaca_dataset.json"
with open(DATA_PATH) as f:
    dataset = json.load(f)
print("Dataset size:", len(dataset))



Dataset size: 200


Building Eambiddings+ FAISS Index

In [35]:
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer("all-MiniLM-L6-v2")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [36]:
import numpy as np
import faiss

# Use only user questions
questions = [d["input"] for d in dataset]

embeddings = embed_model.encode(questions, show_progress_bar=True)

#  normalize for cosine similarity
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

dim = embeddings.shape[1]

# inner product index (cosine similarity)
index = faiss.IndexFlatIP(dim)
index.add(embeddings)

print("FAISS index built:", index.ntotal)



Batches:   0%|          | 0/7 [00:00<?, ?it/s]

FAISS index built: 200


In [37]:
faiss.write_index(index, "/content/drive/MyDrive/bfsi/data/faiss_cosine.index")


Similarity Search Function

In [38]:
def similarity_search(query, threshold=0.70, top_k=3):
    """
    Cosine similarity search
    Returns best match from dataset
    """

    q_emb = embed_model.encode([query])

    # normalize
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    scores, indices = index.search(q_emb, top_k)

    best_score = scores[0][0]
    best_idx = indices[0][0]

    result = {
        "matched": False,
        "response": None,
        "score": round(float(best_score), 3),
        "source": "None"
    }

    if best_score >= threshold:
        result["matched"] = True
        result["response"] = dataset[best_idx]["output"]
        result["source"] = "Dataset"

    return result


Test it

In [39]:
tests = [
    "How do I update my mobile number?",
    "What happens if I miss my EMI?",
    "Tell me about interest rates"
]

for q in tests:
    r = similarity_search(q)
    print("\nQuery:", q)
    print(r)



Query: How do I update my mobile number?
{'matched': True, 'response': 'You may update your registered mobile number through the profile section after completing OTP verification. For assistance, contact support.', 'score': 1.0, 'source': 'Dataset'}

Query: What happens if I miss my EMI?
{'matched': True, 'response': 'Delayed payments may attract late fees and additional interest as per policy. We recommend paying dues before the due date to avoid penalties and credit score impact.', 'score': 1.0, 'source': 'Dataset'}

Query: Tell me about interest rates
{'matched': False, 'response': None, 'score': 0.691, 'source': 'None'}


In [40]:
!pip install -q transformers


Load Model

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL_NAME = "google/flan-t5-base"

print("Loading model...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

print("Model loaded successfully.")


Loading model...


Loading weights:   0%|          | 0/282 [00:00<?, ?it/s]

The tied weights mapping and config for this model specifies to tie shared.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning


Model loaded successfully.


Genration Prompt

In [42]:
def slm_generate(query, max_tokens=80):

    prompt = f"""
You are a BFSI banking assistant.

Rules:
- Answer only about banking, loans, EMI, payments.
- If unsure, say "Please contact customer support".
- Do NOT invent unrelated terms.

Question: {query}
Answer:
"""

    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_tokens
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


Test The Model

In [43]:
print(slm_generate("Explain how EMI is calculated"))


EMI is a percentage of the total amount of money you pay for a loan or credit card.


In [44]:
def answer_query(query):

    # Tier-1: Dataset similarity
    result = similarity_search(query, threshold=0.65)

    if result["matched"]:
        return result["response"], "Dataset"

    # Tier-2: Local model
    generated = slm_generate(query)
    return generated, "SLM"


Test Complete Flow

In [45]:
tests = [
    "How do I update my mobile number?",
    "Explain EMI formula"
]

for q in tests:
    ans, src = answer_query(q)
    print("\nQuery:", q)
    print("Source:", src)
    print("Answer:", ans)



Query: How do I update my mobile number?
Source: Dataset
Answer: You may update your registered mobile number through the profile section after completing OTP verification. For assistance, contact support.

Query: Explain EMI formula
Source: SLM
Answer: EMI is the amount of money that a bank charges to a customer for a loan.


RAG Layer

In [46]:
!pip install -q langchain faiss-cpu sentence-transformers


Build Knowldge Based Document

In [47]:
knowledge_docs = [
"""
EMI Formula:
EMI = P × r × (1+r)^n / ((1+r)^n − 1)
Where:
P = principal loan amount
r = monthly interest rate
n = number of monthly installments
""",

"""
Interest Rates:
Personal loan interest rates depend on customer credit profile.
Rates typically range between 9% to 14% annually.
Exact rate is decided during approval.
""",

"""
Late Payment Policy:
If EMI is not paid on time, late payment charges and penalties may apply.
This may also negatively affect credit score.
""",

"""
Loan Eligibility:
Eligibility depends on income, credit score, employment stability, and repayment history.
""",

"""
Transaction Issues:
Payments may take up to 24 hours to reflect. If not updated, customers should contact support with reference ID.
"""
]


Create FAISS knowledge index

In [48]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

rag_embed = SentenceTransformer("all-MiniLM-L6-v2")

doc_embeddings = rag_embed.encode(knowledge_docs)
doc_embeddings = doc_embeddings / np.linalg.norm(doc_embeddings, axis=1, keepdims=True)

rag_index = faiss.IndexFlatIP(doc_embeddings.shape[1])
rag_index.add(doc_embeddings)

print("RAG knowledge base ready:", rag_index.ntotal, "documents")


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


RAG knowledge base ready: 5 documents


Retrieve relevant chunks

In [49]:
def retrieve_knowledge(query, top_k=2):

    q_emb = rag_embed.encode([query])
    q_emb = q_emb / np.linalg.norm(q_emb, axis=1, keepdims=True)

    scores, indices = rag_index.search(q_emb, top_k)

    contexts = [knowledge_docs[i] for i in indices[0]]

    return "\n".join(contexts)


RAG Generation

In [50]:
def rag_generate(query):

    context = retrieve_knowledge(query)

    prompt = f"""
You are a BFSI assistant.
Answer ONLY using the context below.
Do NOT invent information.

Context:
{context}

Question: {query}
Answer:
"""

    inputs = tokenizer(prompt, return_tensors="pt")

    outputs = model.generate(**inputs, max_new_tokens=100)

    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


Final Router

In [51]:
def answer_query(query):

    # Tier 1 — Dataset
    result = similarity_search(query)
    if result["matched"]:
        return result["response"], "Dataset"

    # Tier 3 — Use RAG for finance/complex queries
    finance_keywords = ["emi", "interest", "rate", "penalty", "policy", "formula", "loan"]

    if any(word in query.lower() for word in finance_keywords):
        return rag_generate(query), "RAG"

    # Tier 2 — fallback SLM
    return slm_generate(query), "SLM"


Test

In [52]:
tests = [
    "How do I update my mobile number?",     # Tier-1
    "Explain EMI formula",                  # Tier-3
    "What is loan eligibility?",            # Tier-3
    "Hello how are you?"                    # Tier-2
]

for q in tests:
    ans, src = answer_query(q)
    print("\nQuery:", q)
    print("Source:", src)
    print("Answer:", ans)



Query: How do I update my mobile number?
Source: Dataset
Answer: You may update your registered mobile number through the profile section after completing OTP verification. For assistance, contact support.

Query: Explain EMI formula
Source: RAG
Answer: EMI = P  r  (1+r)n / ((1+r)n  1)

Query: What is loan eligibility?
Source: RAG
Answer: depends on income, credit score, employment stability, and repayment history

Query: Hello how are you?
Source: SLM
Answer: good


Guardrail Rules

In [53]:
import re

BLOCK_KEYWORDS = [
    "otp",
    "password",
    "pin",
    "card number",
    "account number",
    "cvv",
    "ssn",
    "social security"
]

OUT_OF_DOMAIN = [
    "politics",
    "election",
    "medical",
    "disease",
    "hack",
    "illegal"
]


Guardrail Check Function

In [54]:
def guardrail_check(query):

    q = query.lower()

    # Sensitive info
    for word in BLOCK_KEYWORDS:
        if word in q:
            return False, "Request involves sensitive information and cannot be processed."

    # Out of domain
    for word in OUT_OF_DOMAIN:
        if word in q:
            return False, "This assistant supports only banking and financial queries."

    return True, None


Safe Response Template

In [55]:
def safe_response(message):
    return f"⚠️ {message} Please contact official customer support for assistance."


Final Ruter

In [56]:
def answer_query(query):

    # Step 0 — Guardrails
    safe, msg = guardrail_check(query)

    if not safe:
        return safe_response(msg), "Guardrail"

    # Tier 1 — Dataset
    result = similarity_search(query)

    if result["matched"]:
        return result["response"], "Dataset"

    # Tier 3 — RAG (finance facts)
    finance_keywords = ["emi", "interest", "rate", "penalty", "policy", "loan", "formula"]

    if any(word in query.lower() for word in finance_keywords):
        return rag_generate(query), "RAG"

    # Tier 2 — SLM fallback
    return slm_generate(query), "SLM"


Test Guardrails

In [57]:
tests = [
    "What is my account number?",   # blocked
    "Send me OTP",                 # blocked
    "Explain EMI formula",         # RAG
    "How do I update mobile number?" # Dataset
]

for q in tests:
    ans, src = answer_query(q)
    print("\nQuery:", q)
    print("Source:", src)
    print("Answer:", ans)



Query: What is my account number?
Source: Guardrail
Answer: ⚠️ Request involves sensitive information and cannot be processed. Please contact official customer support for assistance.

Query: Send me OTP
Source: Guardrail
Answer: ⚠️ Request involves sensitive information and cannot be processed. Please contact official customer support for assistance.

Query: Explain EMI formula
Source: RAG
Answer: EMI = P  r  (1+r)n / ((1+r)n  1)

Query: How do I update mobile number?
Source: Dataset
Answer: You may update your registered mobile number through the profile section after completing OTP verification. For assistance, contact support.


In [58]:
!pip install -q gradio


Professional Chat UI

In [61]:
import gradio as gr

def chat_interface(message, history):

    answer, source = answer_query(message)

    response_text = f"{answer}\n\n(Source: {source})"

    history = history or []

    history.append({"role": "user", "content": message})
    history.append({"role": "assistant", "content": response_text})

    return history


with gr.Blocks() as demo:

    gr.Markdown("#  BFSI AI Assistant")
    gr.Markdown("Loan • EMI • Interest • Account Support")

    chatbot = gr.Chatbot(type="messages")

    msg = gr.Textbox(placeholder="Ask your banking question...")

    msg.submit(chat_interface, [msg, chatbot], chatbot)

demo.launch(share=True)


/tmp/ipython-input-772426328.py:22: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(type="messages")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d43de7c36a4aa62fa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
